# Check implementation of Hamberg et al model

In [1]:
import os

import numpy as np
import plotly.graph_objects as go

from model import define_hamberg_model, define_hamberg_population_model

## Define Hamberg model

In [2]:
directory = os.getcwd()
mechanistic_model, parameters = define_hamberg_model()

## Define covariate model (ignoring other sources of IIV)

In [5]:
def get_typical_parameters(age, cyp2c9, vkorc1):
    """
    Returns typical model parameters for patient with input characteristics
    according to Hamberg et al.

    :param age: The age of the patient.
    :type age: int
    :param cyp2c9: One of 5 variants of the CYP2C9 gene:
        ['*1/*1', '*1/*2', '*1/*3', '*2/*2', '*2/*3','*3/*3']
    :type cyp2c9: str
    :param vkorc1: One of 5 variants of the VKORC1 gene:
        ['GG', 'GA', 'AA']
    :type vkorc1: str
    """
    cyp2c9_variants = ['*1/*1', '*1/*2', '*1/*3', '*2/*2', '*2/*3','*3/*3']
    if cyp2c9 not in cyp2c9_variants:
        raise ValueError(
            'The provided CYP2C9 genotype is not among the allowed variants.')
    vkorc1_variants = ['GG', 'GA', 'AA']
    if vkorc1 not in vkorc1_variants:
        raise ValueError(
            'The provided VKORC1 genotype is not among the allowed variants.')

    # Define covariate independent parameters
    effective_volume_central = 14.3         # in L
    gamma = 1.15                            # dimless
    transition_rate_chain_1 = 3 / 28.6      # in 1/h
    transition_rate_chain_2 = 3 / 118.3     # in 1/h

    # Compute clearance
    clearance_per_allele_1 = 0.174          # in L/h
    clearance_per_allele_2 = 0.0879         # in L/h
    clearance_per_allele_3 = 0.0422         # in L/h

    if cyp2c9 == '*1/*1':
        clearance = 2 * clearance_per_allele_1
    if cyp2c9 == '*1/*2':
        clearance = clearance_per_allele_1 + clearance_per_allele_2
    if cyp2c9 == '*1/*3':
        clearance = clearance_per_allele_1 + clearance_per_allele_3
    if cyp2c9 == '*2/*2':
        clearance = 2 * clearance_per_allele_2
    if cyp2c9 == '*2/*3':
        clearance = clearance_per_allele_2 + clearance_per_allele_3
    if cyp2c9 == '*3/*3':
        clearance = 2 * clearance_per_allele_3

    change_per_year = -0.00571              # rel. change centered around 71 y
    clearance *= (1 + change_per_year * (age - 71))

    # Compute EC50 of warfarin
    ec50_per_allele_A = 0.96                # in mg/L
    ec50_per_allele_G = 2.05                # in mg/L

    if vkorc1 == 'GG':
        half_maximal_effect_conc = 2 * ec50_per_allele_G
    if vkorc1 == 'GA':
        half_maximal_effect_conc = ec50_per_allele_G + ec50_per_allele_A
    if vkorc1 == 'AA':
        half_maximal_effect_conc = 2 * ec50_per_allele_A

    parameters = [
        effective_volume_central,
        clearance,
        gamma,
        half_maximal_effect_conc,
        transition_rate_chain_1,
        transition_rate_chain_2
    ]

    return parameters


## Reproduce Figure 3 in Hamberg et al

In [6]:
# Define simulation conditions
thirty_days = 30 * 24
times = np.linspace(0, thirty_days, num=1000)

# I: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GG VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'GG'
doses = [7.6, 3.8, 1.8]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation1 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    # (Model only captures S-warfarin, so need to divide dose by 2)
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose/2, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation1.append(mechanistic_model.simulate(params, times))

# II: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GA VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'GA'
doses = [5.6, 2.8, 1.4]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation2 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose/2, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation2.append(mechanistic_model.simulate(params, times))

# III: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GA VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'AA'
doses = [3.6, 1.8, 0.9]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation3 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose/2, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation3.append(mechanistic_model.simulate(params, times))

# Figure 1: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
times = times / 24
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[0][0],
    name = '*1/*1 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[1][0],
    name = '*2/*2 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[2][0],
    name = '*3/*3 + GG'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 2: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[0][1],
    name = '*1/*1 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[1][1],
    name = '*2/*2 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[2][1],
    name = '*3/*3 + GG'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Warfarin concentration in mg/L',
)
fig.show()

# Figure 3: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[0][0],
    name = '*1/*1 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[1][0],
    name = '*2/*2 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[2][0],
    name = '*3/*3 + GA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 4: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[0][1],
    name = '*1/*1 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[1][1],
    name = '*2/*2 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[2][1],
    name = '*3/*3 + GA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 5: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[0][0],
    name = '*1/*1 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[1][0],
    name = '*2/*2 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[2][0],
    name = '*3/*3 + AA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 6: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[0][1],
    name = '*1/*1 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[1][1],
    name = '*2/*2 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[2][1],
    name = '*3/*3 + AA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()
